# Optimización de un modelo de probabilidad de Default

#### Iván Pérez Maestre, Alejandro García-Montesinos Huerta, Diego González Villar

Como ya se ha dicho, se muestra a continuación el código que hemos desarrollado para buscar una solución de la forma que se ha venido describiendo en el texto anterior.\
Hemos tratado de ir comentando el código todo lo posible para ayudar al máximo en la comprensión de nuestros razonamientos y, aún así, añadimos texto para ir explicando algunos aspectos en los que queramos hacer especial hincapié.

### Cargamos las librerías que vamos a utilizar

In [ ]:
import warnings #Simplemente por comodidad, para ahorrarnos avisos que no aportan nada
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import argrelextrema #Útil para encontrar extremos relativos
from geneticalgorithm import geneticalgorithm as ga  #Se van a usar métodos genéticos para la minimización. Esta librería en particular

### Cargamos el dataset
Un detalle a comentar es que la empresa nos ha proporcionado dos datasets. En todo momento hemos trabajado con el primero (el más grande) aunque, como se mostrará, el código es igualmente válido para cualquier otro dataset teniendo en cuenta que hay que cargarlo evidentemente.

In [ ]:
#Cargamos el dataset principal y mapping como DataFrame's de Pandas

path_dataset = ?? #Copiar dirección donde se tenga el dataset
main_dataset_df = pd.read_csv(path_dataset,";")

path_mapping = ?? #Copiar dirección donde se tenga el mapping rating-PD
mapping_df = pd.read_csv(path_mapping,";")

### Definimos a continuación la función que buscamos minimizar
La llamamos J por motivos evidentes:


In [ ]:
def J(X): #X es un array (el de los 'min score' que buscamos)
    
    X = np.sort(X) #Hacemos esto para que el algoritmo genético, que trabaja con arrays aleatorios y por tanto, a priori, desordenados...
    #...no se vea afectado por la falta de monotonía, es decir, demos el input que demos, lo primero es ordenar el array de 'min score'.
        
    #Construimos la tabla que queremos "rellenar", es decir, la de ratings con sus PD, LRA DR, número de observaciones, etc.
    global tabla #La creamos como variable global para poder visualizarla después
    tabla = pd.DataFrame({'rating' : mapping_df['rating'], 'min score' : X, 'obs':0, 'PD scale' : mapping_df['pd_scale'],
                      'LRA DR':0})
    
    #Añadimos una columna al main_dataset_df de cara a poner ratings
    global main_dataset_df
    main_dataset_df['rating'] = 0 #Simplemente porque será útil para cálculos posteriores el tener en este dataset el rating de cada préstamo.
    
    #Bucle para contar el número de préstamos de cada rating teniendo en cuenta la reestructuración
    i = 1
    while i < len(X):
        
        #Contamos las obs (número de préstamos en cada rating)
        tabla['obs'][i-1] = tabla['obs'][i-1] + len(main_dataset_df[(main_dataset_df['score']>=X[i-1]) & (main_dataset_df['score']<X[i]) & 
                                               (main_dataset_df['b_reestructured'] == 0)])
        tabla['obs'][i] = tabla['obs'][i] + len(main_dataset_df[(main_dataset_df['score']>=X[i-1]) & (main_dataset_df['score']<X[i]) & 
                                               (main_dataset_df['b_reestructured'] == 1)]) #Tenemos en cuenta que si está reestructurado, la obser...
                                                                                            #...vación va en el rating posterior como se ha explicado.
        
        #A continuación se va escribiendo que rating tiene cada préstamo. Para esto simplemente se "sacan" los préstamos con score entre dos "min score"
        #y se les asocia el valor de rating que corresponda.
        main_dataset_df.loc[(main_dataset_df['score']>=X[i-1]) & (main_dataset_df['score']<X[i]) & 
                            (main_dataset_df['b_reestructured'] == 0),'rating'] = tabla['rating'][i-1]
        main_dataset_df.loc[(main_dataset_df['score']>=X[i-1]) & (main_dataset_df['score']<X[i]) & 
                            (main_dataset_df['b_reestructured'] == 1),'rating'] = tabla['rating'][i] #Se vuelve a tener en cuenta el reestructurado.
        
        i = i + 1
        
    #Hasta aquí, no se ha tenido el en cuenta el último rating, pero lo introducimos usando que ahora i ya tiene el último índice (fuera del...
    #...bucle para así evitar salirnos de la longitud del array):
    tabla['obs'][i-1] = tabla['obs'][i-1] + len(main_dataset_df[main_dataset_df['score']>=X[i-1]])
    main_dataset_df.loc[main_dataset_df['score']>=X[i-1],'rating'] = tabla['rating'][i-1]
    #En el último rating están aquellos préstamos que tengan simplemente un score mayor al último valor de 'min score'
    
    #Ya tenemos la tabla preparada para hacer cálculos:
    global dataset_agrupado #Simplemente vamos a ir agrupando por rating y snapshot de cara a hacer los sumatorios necesarios.
    
    dataset_agrupado = main_dataset_df.groupby(by = ['rating','snapshot']).agg({'b_default' : ['sum','count']})
    #Para rating y snapshot fijado contamos el número de préstamos que hay y además sumamos los b_default
    dataset_agrupado['DR'] = dataset_agrupado.iloc[:,0] / dataset_agrupado.iloc[:,1] #No hay problema en la división, por construcción no se divide entre 0
    #De esta forma ya tenemos el DR que queríamos.

    #Resta sumar los DR en todos los snapshots dado un mismo rating y dividir entre el número de dichos snapshots
    LRA_DR = dataset_agrupado.groupby('rating').DR.agg(sum) / dataset_agrupado.groupby('rating').DR.agg('count')
    LRA_DR = pd.DataFrame(LRA_DR) #Creo DataFrame por comodidad
    #Por explicarlo, en el numerador agrupamos de nuevo solo por rating y sumamos los DR y después en el denominador simplemente dividimos
    
    #Introducimos el LRA_DR calculado en la columna que habíamos reservado en nuestra tabla:
    tabla = tabla.set_index('rating') #Simplemente hago esto para que se reconozcan bien los índices al meter la nueva columna
    tabla['LRA DR'] = LRA_DR
    tabla = tabla.reset_index() #Deshago lo de los índices
    #Se podría observar que, si no hay observaciones en un cierto rating, el LRA DR asociado es NaN, lo cual simplemente hace ver...
    #...que en efecto no hay observaciones y no se puede calcular dicho LRA DR. Para los cálculos siguientes se tendrá en cuenta esto y evitar los...
    #...posibles problemas que acarree.
    
    #UNA VEZ TENEMOS LA TABLA QUE NOS INTERESA CREADA, HAY QUE INTRODUCIR LA FUNCIÓN J A MINIMIZAR. Usamos el razonamiento explicado:
    Err_rel = abs(tabla['PD scale'] - tabla['LRA DR'])/tabla['PD scale']#Esencialmente esa es la función a penalizar a la que añadimos penalización
    
    #Penalización si no hay unimodalidad:
    a = np.array(tabla['obs'])
    a = np.concatenate((np.array([-1,-1]),a)) ##Introducimos dos valores negativos al principio con el objetivo de que si las observaciones...
    #... empiezan con un máximo, la función lo cuente como extremo relativo y se cumpla la unimodalidad. Simplemente es por cómo funciona argrelextrema
    max_rel = argrelextrema(a, np.greater) #Lo único que nos importa realmente es el número de extremos relativos
    if len(max_rel[0]) != 1: #Buscamos que haya un único máximo (unimodalidad)
        return float('inf') #Directamente llevamos nuestra función de coste a infinito y no seguimod porque no se cumple esta condición.
    
    #Tampoco queremos que haya mínimos:
    b = np.array(tabla['obs'])
    b = np.concatenate((np.array([-1,-1]),b)) ##Introducimos dos valores negativos al principio con el objetivo de que si las observaciones...
    #... empiezan con un máximo, la función lo cuente como extremo relativo y se cumpla la unimodalidad
    min_rel = argrelextrema(b, np.less) #Lo único que nos importa realmente es el número de extremos relativos
    if len(min_rel[0]) != 0: #Buscamos que haya un único máximo (unimodalidad)
        return float('inf')
    #Esto se hace porque, de primeras, solo introdujimos la penalización si había más de un máximo pero vimos que podía darse la situación de que...
    #...hubiera solo un máximo pero no unimodalidad. De esta forma aseguramos dicha tendencia.
    
    
    #Penalización si hay porcentaje alto en un mismo rating
    obs_rel = tabla['obs'] / len(main_dataset_df) #Número relativo de observaciones
    if np.array_equal(np.array(obs_rel < 0.35),np.array([True]*len(tabla))) == False: #Nos aseguramos que como mucho la concentración en un mismo rating sea del 40%
        return float('inf')
    #Aunque la condición pueda parecer en cierto modo extraña, lo que se está diciendo es "sácame" un array de variables tipo booleanas con...
    #... la condición de que la concentración sea menos de 0.35. Se cumple lo que queremos si dicho array es igual a un array de todo "True".
    #Si no pasa esto, es decir, al menos un rating tiene mayor concentración con lo que en ese rating se tiene un "False", se lleva la función...
    #...de coste a infinito.
    
    #Penalización si hay demasiados ratings sin observaciones
    if len(tabla[tabla['obs'] == 0])/len(tabla) > 2/len(tabla):
        return float('inf')
    #Simplemente hemos valorado que parece sensato pedir que no haya más de dos ratings vacíos dado que tampoco tenemos tantos.
    
    return np.nansum(Err_rel) #Tenemos en cuenta que puede haber valores con NaN y usamos numpy para tratar esos valores como 0
    #De esta forma tan sencilla ya tenemos introducida la primera parte de la función a minimizar
    

### Observaciones sobre esta función:
#### 1º observación (y más importante)
Los cálculos se hacen exclusivamente con funciones internas de la librería pandas en el dataset. Esto nos parece importante recalcarlo porque en una primera aproximación, introdujimos un bucle for que recorría todo el dataset y sí, funcionaba igual, pero el tiempo gastado para calcular la función de coste para un array cualquiera de input era demasiado elevado. Esto, si se hace para una sola ejecución, puede no tener demasiada importancia, pero cuando el algoritmo genético lo haga miles de veces, entonces ya la cosa cambia. Este paso nos supuse la clave para avanzar lo más rápido posible y ganamos mucho tiempo.
#### 2º observación
El algoritmo genético, como bien se comenta en el código, va a probar con poblaciones cuyos individuos (arrays) están, con toda probabiliad, desordenados y nosotros buscamos monotonía. Esto simplemente se consigue haciendo que lo primero que hace nuetsra función sea ordenar el array y que así todos los cálculos se hagan en base a un array de "min score" ordenado. Así, nos podemos olvidar de la monotonía.
#### 3º observación
Incidimos aquí un poco más en por qué se decidió llevar a infinito la J si había mucha concentración de préstamos en un mismo rating o muchos de ellos vacíos.
Aunque todavía no hemos mostrado el algoritmo genético, ya no es más que un pequeño código y no va a aportar más a esta explicación. Simplemente se debe a que, como se ha dicho en el PDF, esta no fue nuestra primera idea y probamos otras dos opciones, pero los resultados que nos sacaba la función no eran como los deseados. Es decir, dado que el algoritmo genético tiene una buena parte de "azar" en el sentido de los individuos aleatorios que genera, es muy posible que genere arrays que llevan a, por ejemplo, mucha concentración en un mismo rating, e ir evolucionando estos llegando al final a una solución que no es la deseada.
Con este proceso lo que se hace es ser muy estricto con que se cumplan las condiciones que queramos y, aunque el algoritmo pueda tardar más en encontrar arrays que lleven a funciones de coste menores que infinito, así encontras soluciones óptimas.

### Introducción del algoritmo genético
Es importante darse cuenta de que en el algoritmo se tienen que dar los extremos de los intervalos en los que se desean encontrar los elementos del array (por ello damos dos algoritmos, uno con extremos en el caso del dataset grande ya que en ese el mapping rating-PD es más largo y por tanto hay que añadir más intervalos y otro para el pequeño).\
Además, hay que tener en cuenta otro aspecto sobre los extremos de los intervalos en los que buscar y es que, al menos uno (el primero cuando se ordene) tiene que tener un valor menor que el menor rating que haya en el dataset. Si no se impone esta condición podría ocurrir (y de hecho nos ocurrió) que el primer "min score" sea mayor que el score mínimo del dataset y se dejen préstamos fuera cosa que, evidentemente, no queremos. De esta forma conseguimos que no se nos escape ningún préstamo.
El resto simplemente buscamos entre el score mínimo y el máximo, nos parece algo lógico.

Por otro lado, el algoritmo tiene una serie de parámetros. En general hemos tomado los valores que la librería pone por defecto (ya que hemos probado a cambiarlos pero no hemos visto cambios significativos), pero sí cabe destacar la importancia de los parámetros "population_size" y "max_num_iteration". El primero hace referencia a, precisamente, el tamaño de la población con la que se trabaja y el segundo es cuántas iteraciones queremos que se realicen si no se agota el tiempo (tiempo que ponemos alto para dar tiempo al algoritmo). Estos son importantes ya que hay que encontrar el cierto modo el equilibrio entre no poner poblaciones muy grandes o demasiadas iteraciones pues se aumentaría mucho el tiempo que tardaría el algoritmo (aunque los resultados fueran mejores) y no tener poblaciones demasiado escasas pues podría llevar a que el algoritmo no consiga bajar de infinito la función de coste.

Sobre esto último es evidente que, con lo restrictivos que hemos sido, es de esperar que los primeros valores de J que vaya encontrando el algoritmo sean infinito pues lo más probable es que los array que genere aleatoriamente en primer lugar, no cumplan con todos nuestros requisitos. De hecho este es el mayor problema de nuestra solución a nuestro juicio, cosa que discutiremos más adelante.

#### Dataset y mapping rating-PD grandes

In [ ]:
import warnings
warnings.filterwarnings('ignore')
#Ahora simplemente aplicamos el algoritmo genético a nuestra función J(X)
varbound=np.array([[min(main_dataset_df['score'])-0.5,min(main_dataset_df['score'])-0.1], [min(main_dataset_df['score']),max(main_dataset_df['score'])],
         [min(main_dataset_df['score']),max(main_dataset_df['score'])],[min(main_dataset_df['score']),max(main_dataset_df['score'])],
         [min(main_dataset_df['score']),max(main_dataset_df['score'])],[min(main_dataset_df['score']),max(main_dataset_df['score'])],
         [min(main_dataset_df['score']),max(main_dataset_df['score'])],[min(main_dataset_df['score']),max(main_dataset_df['score'])],
         [min(main_dataset_df['score']),max(main_dataset_df['score'])],[min(main_dataset_df['score']),max(main_dataset_df['score'])],
         [min(main_dataset_df['score']),max(main_dataset_df['score'])],[min(main_dataset_df['score']),max(main_dataset_df['score'])],
         [min(main_dataset_df['score']),max(main_dataset_df['score'])],[min(main_dataset_df['score']),max(main_dataset_df['score'])]])

algorithm_param = {'max_num_iteration': ??,\ #Escribir número de iteraciones máximo
                   'population_size':??,\ #Escribir tamaño de la población
                   'mutation_probability':0.1,\
                   'elit_ratio': 0.01,\
                   'crossover_probability': 0.5,\
                   'parents_portion': 0.3,\
                   'crossover_type':'uniform',\
                   'max_iteration_without_improv':None}

model=ga(function=J,dimension=len(mapping_df),variable_type='real',variable_boundaries=varbound,function_timeout = 10000,
        algorithm_parameters=algorithm_param)
model.run()

#### Dataset y mapping rating-PD pequeños
Simplemente cambia el número de intervalos que hay que dar pues el mapping es más pequeño.

Hay que tener en cuenta que para hacerlo hay que cargar primero el nuevo dataset y mapping.

In [ ]:
import warnings
warnings.filterwarnings('ignore')
#Ahora simplemente aplicamos el algoritmo genético a nuestra función J(X)
varbound=np.array([[min(main_dataset_df['score'])-0.5,min(main_dataset_df['score'])-0.1], [min(main_dataset_df['score']),max(main_dataset_df['score'])],
         [min(main_dataset_df['score']),max(main_dataset_df['score'])],[min(main_dataset_df['score']),max(main_dataset_df['score'])],
         [min(main_dataset_df['score']),max(main_dataset_df['score'])],[min(main_dataset_df['score']),max(main_dataset_df['score'])],
         [min(main_dataset_df['score']),max(main_dataset_df['score'])],[min(main_dataset_df['score']),max(main_dataset_df['score'])],
         [min(main_dataset_df['score']),max(main_dataset_df['score'])],[min(main_dataset_df['score']),max(main_dataset_df['score'])],
         [min(main_dataset_df['score']),max(main_dataset_df['score'])],[min(main_dataset_df['score']),max(main_dataset_df['score'])]])

algorithm_param = {'max_num_iteration': ??,\ #Introducir iteraciones
                   'population_size':??,\ #Introducir tamaño de la población
                   'mutation_probability':0.1,\
                   'elit_ratio': 0.01,\
                   'crossover_probability': 0.5,\
                   'parents_portion': 0.3,\
                   'crossover_type':'uniform',\
                   'max_iteration_without_improv':None}

model=ga(function=J,dimension=len(mapping_df),variable_type='real',variable_boundaries=varbound,function_timeout = 10000,
        algorithm_parameters=algorithm_param)
model.run()

### Próximos pasos:
Hasta aquí está todo el código que se necesita y con el mayor detalle que se ha podido. El lector puede probarlo por sí mismo cargando el dataset que quiera (podría ser cualquiera con la misma estructura que el que se nos ha dado).

A partir de aquí ya se vuelve al documento de texto con el que se empezó y lo último que se pretende hacer es mostrar resultados para cada dataset a los que hemos llegado nosotros usando este código donde por un lado vamos a mostrar los mejores (y por tanto los mejores score-to-rating mappings, que es lo que se nos pedía) y por otro vamos a variar el tamaño de la población y número de iteraciones para llegar a soluciones de manera más rápida.\
También se va a hacer un pequeño análisis de los pros y contras que vemos de nuestra solución y con eso concluirá todo.